In [3]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, LeakyReLU, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.initializers import GlorotUniform

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-12-16 19:24:01.719584: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
TIMEFRAMES = [14, 30, 90, 180, 365]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['OHLC', 'ColoredOHLC', 'Line', 'AlgoTrading']

labels = pd.read_csv('labels.csv')

In [4]:
# Find bounding boxes of all 4 image sizes
def find_bounding_box(img_path):
    # Load the image
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find all non-black pixels
    coords = cv2.findNonZero(gray)
    
    # Find the bounding box of those pixels
    x, y, w, h = cv2.boundingRect(coords)
    print(x, y, w, h)

# Define the paths to your images
image_paths = [
    'images/Line/AI&Robotics_2019-01-15 00:00:00_14.png',
    'images/Line/AI&Robotics_2019-01-24 00:00:00_30.png',
    'images/Line/AI&Robotics_2019-03-28 00:00:00_90.png',
    'images/Line/AI&Robotics_2019-06-20 00:00:00_180.png', 
    'images/Line/AI&Robotics_2019-12-26 00:00:00_365.png'   
]

# Crop each image and save the result
for path in image_paths:
    cropped_image = find_bounding_box(path)

80 100 85 120
80 100 132 120
80 100 226 120
80 100 414 120
80 100 602 120


In [3]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
        elif number == 365:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+602]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)


In [5]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_3206/666818149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')


In [6]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [7]:
# Divide data based on timeframes
labels_dict = {
    "14" : labels[labels['Image'].str.contains('_14_')],
    "30" : labels[labels['Image'].str.contains('_30_')],
    "90" : labels[labels['Image'].str.contains('_90_')],
    "180" : labels[labels['Image'].str.contains('_180_')],
    "365" : labels[labels['Image'].str.contains('_365_')]
    }

In [8]:
for key in labels_dict:
    # Convert the Image_Array column to a numpy array
    labels_dict[key]['Image_Array'] = np.array(labels_dict[key]['Image_Array']) / 255.0
    # Shuffle the DataFrame
    #labels_dict[key] = shuffle(labels_dict[key], random_state=42).reset_index(drop=True)


/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_3206/2776856819.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_dict[key]['Image_Array'] = np.array(labels_dict[key]['Image_Array']) / 255.0


In [9]:
def create_cnn_model(num_blocks, input_shape=()):
    model = Sequential()
    
    current_shape = input_shape
    for i in range(num_blocks):
        # Add Convolutional layer
        filters = 64 * (2 ** i)
        model.add(Conv2D(filters, kernel_size=(5, 3), dilation_rate=(1, 1), padding='same',
                         input_shape=current_shape))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.01))
        model.add(MaxPooling2D(pool_size=(2, 1)))

        # Update current_shape for the next layer
        current_shape = (current_shape[0] // 2, current_shape[1] // 3, filters)

    # Calculate the number of neurons for the fully connected layer
    flatten_size = current_shape[0] * current_shape[1] * current_shape[2]
    
    # Flatten the output of the last CNN block
    model.add(Flatten())

    # Fully connected layer with Dropout
    model.add(Dense(flatten_size, activation='relu', kernel_initializer=GlorotUniform()))
    model.add(Dropout(0.5))

    # Output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model with binary cross-entropy loss and Adam optimizer
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [11]:
def create_cnn_model(num_blocks, initial_filters=64, input_shape=()):
    model = Sequential()

    for i in range(num_blocks):
        # Adjust the number of filters
        filters = initial_filters * (2 ** i)

        # Set dilation rate and strides
        strides = (1, 3)

        # Add Convolutional layer
        model.add(Conv2D(filters, kernel_size=(5, 3), strides=strides, padding='same',
                         input_shape=input_shape if i == 0 else model.output_shape[1:]))

        # Add LeakyReLU layer
        model.add(LeakyReLU(alpha=0.01))

        # Add MaxPooling layer only if dimensions allow
        if model.output_shape[1] >= 2 and model.output_shape[2] >= 2:
            model.add(MaxPooling2D(pool_size=(2, 2)))

    # Add Flatten and Dense layers for final prediction
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))  # Example Dense layer, adjust as needed
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Adjust the final layer as per your classification/regression needs

    # Compile the model with binary cross-entropy loss and Adam optimizer
    model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [12]:
evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F2_Score'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:
                print(f"Predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")
                
                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(img_type)) & (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
                
                # Create and compile your CNN model based on timeframe
                if timeframe == 14:
                    model = create_cnn_model(2, input_shape=(120, 85, 1))
                elif timeframe == 30:
                    model = create_cnn_model(3, input_shape=(120, 132, 1))
                elif timeframe == 90:
                    model = create_cnn_model(4, input_shape=(120, 226, 1))
                elif timeframe == 180:
                    model = create_cnn_model(4, input_shape=(120, 414, 1))
                elif timeframe == 365:
                    model = create_cnn_model(4, input_shape=(120, 602, 1))
                
                X = np.array(data['Image_Array'].tolist())
                y = data['Label']
                X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
                
                # Train the model
                model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))
                
                # Evaluate the model
                y_pred = model.predict(X_val)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)
                
                accuracy = accuracy_score(y_val, y_pred_binary)
                precision = precision_score(y_val, y_pred_binary)
                recall = recall_score(y_val, y_pred_binary)
                f2_score = fbeta_score(y_val, y_pred_binary, beta=2)
                
                print("Evaluation Metrics:")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F2 Score: {f2_score}")
                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': accuracy,
                    'Precision': precision,
                    'Recall': recall,
                    'F2_Score': f2_score
                }, ignore_index=True)
                
                # Save the model
                model.save(f"models/{img_type}_{timeframe}_{prediction}.h5")
                print(f"Model saved as {img_type}_{timeframe}_{prediction}.h5")
                print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'evaluation_scores.csv'.")

Predicting 5 days ahead using OHLC images with 14 days timeframe.
Epoch 1/10
20/20 [==============================] - 3s 103ms/step - loss: 8.0600 - accuracy: 0.4847 - val_loss: 4.1262 - val_accuracy: 0.5993
Epoch 2/10
20/20 [==============================] - 2s 91ms/step - loss: 4.8664 - accuracy: 0.6103 - val_loss: 1.9214 - val_accuracy: 0.6442
Epoch 3/10
20/20 [==============================] - 2s 88ms/step - loss: 3.5653 - accuracy: 0.6377 - val_loss: 1.6043 - val_accuracy: 0.7079
Epoch 4/10
20/20 [==============================] - 2s 89ms/step - loss: 2.3877 - accuracy: 0.7085 - val_loss: 1.3145 - val_accuracy: 0.7416
Epoch 5/10
20/20 [==============================] - 2s 92ms/step - loss: 2.1855 - accuracy: 0.7150 - val_loss: 1.2659 - val_accuracy: 0.7640
Epoch 6/10
20/20 [==============================] - 2s 95ms/step - loss: 1.5274 - accuracy: 0.7279 - val_loss: 1.0378 - val_accuracy: 0.7640
Epoch 7/10
20/20 [==============================] - 2s 101ms/step - loss: 1.2889 - accu

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11/11 [==============================] - 3s 183ms/step - loss: 3.4302 - accuracy: 0.5559 - val_loss: 1.3667 - val_accuracy: 0.5175
Epoch 2/10
11/11 [==============================] - 2s 170ms/step - loss: 2.9374 - accuracy: 0.5347 - val_loss: 1.6616 - val_accuracy: 0.5664
Epoch 3/10
11/11 [==============================] - 2s 184ms/step - loss: 2.2165 - accuracy: 0.5861 - val_loss: 1.2462 - val_accuracy: 0.5874
Epoch 4/10
11/11 [==============================] - 2s 177ms/step - loss: 1.3446 - accuracy: 0.6254 - val_loss: 0.9821 - val_accuracy: 0.6154
Epoch 5/10
11/11 [==============================] - 2s 176ms/step - loss: 1.3861 - accuracy: 0.6254 - val_loss: 1.0137 - val_accuracy: 0.6224
Epoch 6/10
11/11 [==============================] - 2s 171ms/step - loss: 1.2494 - accuracy: 0.6284 - val_loss: 0.8930 - val_accuracy: 0.6364
Epoch 7/10
11/11 [==============================] - 2s 176ms/step - loss: 0.9061 - accuracy: 0.6918 - val_loss: 0.8148 - val_accuracy: 0.6364
Epoch 8/10
11/11 

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
5/5 [==============================] - 3s 389ms/step - loss: 1.2106 - accuracy: 0.5849 - val_loss: 0.8369 - val_accuracy: 0.5362
Epoch 2/10
5/5 [==============================] - 2s 335ms/step - loss: 0.9205 - accuracy: 0.5723 - val_loss: 0.7902 - val_accuracy: 0.5652
Epoch 3/10
5/5 [==============================] - 2s 338ms/step - loss: 0.8137 - accuracy: 0.6038 - val_loss: 0.7472 - val_accuracy: 0.5797
Epoch 4/10
5/5 [==============================] - 2s 350ms/step - loss: 0.7002 - accuracy: 0.6792 - val_loss: 0.7306 - val_accuracy: 0.5942
Epoch 5/10
5/5 [==============================] - 2s 347ms/step - loss: 0.6212 - accuracy: 0.7107 - val_loss: 0.7089 - val_accuracy: 0.6087
Epoch 6/10
5/5 [==============================] - 2s 351ms/step - loss: 0.5398 - accuracy: 0.7547 - val_loss: 0.6817 - val_accuracy: 0.5942
Epoch 7/10
5/5 [==============================] - 2s 348ms/step - loss: 0.5247 - accuracy: 0.7358 - val_loss: 0.6656 - val_accuracy: 0.5942
Epoch 8/10
5/5 [====

...

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
3/3 [==============================] - 3s 578ms/step - loss: 1.9182 - accuracy: 0.6716 - val_loss: 1.0163 - val_accuracy: 0.5862
Epoch 2/10
3/3 [==============================] - 2s 487ms/step - loss: 1.1838 - accuracy: 0.7463 - val_loss: 1.2858 - val_accuracy: 0.6552
Epoch 3/10
3/3 [==============================] - 2s 535ms/step - loss: 1.2448 - accuracy: 0.7463 - val_loss: 1.3960 - val_accuracy: 0.6207
Epoch 4/10
3/3 [==============================] - 2s 524ms/step - loss: 1.3003 - accuracy: 0.7164 - val_loss: 1.2972 - val_accuracy: 0.6552
Epoch 5/10
3/3 [==============================] - 2s 509ms/step - loss: 1.0657 - accuracy: 0.6866 - val_loss: 1.2372 - val_accuracy: 0.6552
Epoch 6/10
3/3 [==============================] - 2s 512ms/step - loss: 0.8335 - accuracy: 0.6866 - val_loss: 1.2458 - val_accuracy: 0.6552
Epoch 7/10
3/3 [==============================] - 2s 528ms/step - loss: 0.7929 - accuracy: 0.7463 - val_loss: 1.2439 - val_accuracy: 0.6207
Epoch 8/10
3/3 [====

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
3/3 [==============================] - 3s 568ms/step - loss: 2.6231 - accuracy: 0.4478 - val_loss: 1.0319 - val_accuracy: 0.5517
Epoch 2/10
3/3 [==============================] - 2s 476ms/step - loss: 1.5711 - accuracy: 0.6418 - val_loss: 1.0095 - val_accuracy: 0.6552
Epoch 3/10
3/3 [==============================] - 2s 495ms/step - loss: 1.8635 - accuracy: 0.5672 - val_loss: 0.9194 - val_accuracy: 0.6552
Epoch 4/10
3/3 [==============================] - 2s 538ms/step - loss: 1.5905 - accuracy: 0.6567 - val_loss: 0.8786 - val_accuracy: 0.6897
Epoch 5/10
3/3 [==============================] - 2s 547ms/step - loss: 1.5902 - accuracy: 0.5821 - val_loss: 0.8859 - val_accuracy: 0.7241
Epoch 6/10
3/3 [==============================] - 2s 497ms/step - loss: 1.0928 - accuracy: 0.6866 - val_loss: 0.9575 - val_accuracy: 0.7241
Epoch 7/10
3/3 [==============================] - 2s 515ms/step - loss: 1.6760 - accuracy: 0.5224 - val_loss: 0.8592 - val_accuracy: 0.7241
Epoch 8/10
3/3 [====

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
